# Matthew Manno
# is360 Final Project

I made this notebook to help me with a project that I have at work. I am a construction scheduler and I work with a program that stores information about projects in a relational database. The program allows you to import information about a schedule into it using Microsoft Excel files. The two types of information I am interested in importing into the program are:

1. __Activity/Task Information :__ Activity information contains general information about the main constituents of a project, the activities/tasks. Each activity describes a portion of work on a project. The columns that make up an activity are:
    
    1. __ID :__ This is a unique alpha numeric identifier for an activity
    
    2. __Status :__ This tells the program the current activity's status. If it has started, or if it has completed. All of the activities we will be making will have a status of "Not Started". This field is basically worthless, but the program requieres it when you are importing an activitiy.
    
    3. __WBS :__ The wbs code is a code that tells the program where to place the activity in the project's WBS hierachy. All activities will be imported into the same location in the WBS hiearchy. This field is basically worthless, but the program requieres it when you are importing an activitiy.
    
    4. __act_name :__ This field is a human readable field that contains (2)two pieces of information.<BR>    1: "prefix" consisting of a description of the work represented by the actvity. ie: "Erect Masonry Walls"<BR>    2: A "suffix" consisting of where the work will be performed. ie: " @ 3rd Floor"
    
    5. __duration :__ This is an integer it represents how many days an activity will take.
    
    6. __start :__ This field tells us when the activity will eventually _start_, based on information I recieved from my client. As this project is currently underway, not all activities will have a start date.
    
    7. __finish :__ This field tells us when the activity will eventually _finish_, based on information I recieved from my client. As this project is currently underway, not all activities will have a finish date.
    
    8. __floor_id :__ This field corresponds with another table in the database that tells us what floor the activity is on. ie: the 4th floor. This field is imported to the program as a string. Because not all floors are nummeric or even floors. For instance "Function Room 1" This "floor" has a code of FR1.<BR><BR>
2. __Relationship/Logic Information :__ Relationship information consists of the realationship / logic information of a project. I will go into detail on this information below.

In [1]:
# Import numpy
import numpy as np
# Import Pandas
import pandas as pd
# import the requests library to send http requests to the NYT API
import requests
# import json library for parsing JSON and converting JSON to python dictionaries
import json as json

#set some pandas options controling output format
pd.set_option('display.notebook_repr_html',True) # output as flat text and not HTML
pd.set_option('display.max_rows', None) # this is the maximum number of rows we will display
pd.set_option('display.max_columns', None) # this is the maximum number of rows we will display

In [2]:
# There are two types of activities that we can create with this notebook. One are activities 
# that we have start and finish data for, and ones that we do not. strStandardActivities will hold
# the file string for activities which we do not have dates for.
strStandardActivities = 'input/Import_Activities-Standard_Floors.csv'

# strSourceActivities holds the file string for activities for activities that we do have dates for.
strSourceActivities = 'input/Import_5Beek_Amen-Activities.csv'
# strSourceLogic contains a template for our activity logic. This will allow us to create logic 
# quickly for every activity on a floor.
strSourceLogic = 'input/Import_5Beek_Amen-Logic.csv'

# This is the file we will output our activity information to.
strActivityFileOutput = 'output/Export_Activities_From_Pandas.csv'
# This is the file we will output our logic information to.
strLogicFileOutput = 'output/Export_Logic_From_Pandas.csv'

# the string to cache the WBS code for activities we are adding. They will all have the same WBS code
strWBSCode = 'BH.BL-4.115 Nassau1.Floors.4'
# the string to cache the activity status information. All activities will be "Not Started".
strStatus = 'Not Started'

In [3]:
# this function takes a template of activities and duplicates them the specified number of times.
# intStart: represents the floor to start creating activities for.
# intFinish: represents the floor to stop creating activities for.
def setActivitiesFromTemplate(intStart, intFinish):
    # import the template for the activities we will be basing the schedule off of.
    dfNew = pd.read_csv(strStandardActivities,dtype={'ID': str})
    # declare the dataframe that will hold our tasks.
    dfNewTasks = pd.DataFrame(columns=['id', 'status', 'wbs', 'act_name', 'duration', 'floor_id'])

    # Within the loop below we are formatting the activity information for importing.
    #    id: has 3 parts:
    #         1. Starts with the string "NA." 
    #         2. The current loop's integer as a 2 digit number with preceeding zeroes. 
    #            This represents the floor the activity is on.
    #         3. Ends with the ID code for the current activity as a 3 digit number with
    #            preceeding zeroes. This represents the activities unique ID.
    #            For example:
    #        "NA." + "02" + "990" => NA.10990
    #     act_name: holds the activity description is consists of the activity description
    #     the string " @ Floor " and the integer of the floor it's on cast as a string.
    #     For example:
    #        "Build CMU Walls" + " @ Floor " + 2 => "Build CMU Walls @ Floor 2"
    
    # for the range the user stipulated...
    for i in range(intStart, intFinish):
        # cache the new dataframe with the new string formatting
        dfCache = pd.DataFrame({
            'id' : 'NA.' + str(i).zfill(2) + dfNew['ID'].str.zfill(3), \
            'status' : strStatus, \
            'wbs' : strWBSCode, \
            'act_name' : dfNew['Name'] + ' @ Floor ' + str(i), \
            'duration' : dfNew['Duration'], \
            'floor_id' : str(i).zfill(2)})
        
        # combine the new dataframe with the exiting one.
        dfNewTasks = pd.DataFrame(pd.concat([dfNewTasks, dfCache]))
                
    # reset the index of the new dataframe
    dfNewTasks.reset_index(inplace=True, drop=True) 
    
    # Reorder the columns to make import easier.
    dfNewTasks = dfNewTasks[['id', 'status', 'wbs', 'act_name', 'duration', 'floor_id']]

    # Return the formatted activity dataframe.
    return dfNewTasks

In [4]:
# This portion of the notebook is for activities that we have "start" and "finish" dates 
# supplied by the client. Not all activities will have start or finish dates.
# import our activity source file
dfSource = pd.read_csv(strSourceActivities,dtype={'ID': str, 'FloorID' : str})
# format the ID string to be 3 characters long with leading zeroes
dfSource['ID'] = dfSource['ID'].str.zfill(3)
# format the FloorID string to be 2 characters long with leading zeroes
dfSource['FloorID'] = dfSource['FloorID'].str.zfill(2)
# preview the dataframe
dfSource.head(1)

,ID,Name,Duration,AS,AF,FloorID
0,290,Final Paint @ Function Room 1,4,NaN,NaN,FR1


In [5]:
# create a new dataframe that contains our activity data for export formatted correctly
# This is simmilar to the formatting outlined in the setActivitiesFromTemplate function.
dfActivities = pd.DataFrame({
    'id' : 'NA.' + dfSource['FloorID'] + dfSource['ID'], \
    'status' : strStatus, 'wbs' : strWBSCode, \
    'act_name' : dfSource['Name'], 'duration' : dfSource['Duration'], \
    'start' : dfSource['AS'], 'finish' : dfSource['AF'], \
    'floor_id' : dfSource['FloorID']})
# preview the dataframe
dfActivities.head(1)

,act_name,duration,finish,floor_id,id,start,status,wbs
0,Final Paint @ Function Room 1,4,NaN,FR1,NA.FR1290,NaN,Not Started,BH.BL-4.115 Nassau1.Floors.4


In [6]:
# Reorder the columns to make import easier.
dfActivities = dfActivities[['id', 'status', 'wbs', 'act_name', 'duration', 'start', 'finish', 'floor_id']]
# sort the dataframe by 'id'
dfActivities.sort_values('id', inplace=True)
# export the dataframe, without the index
dfActivities.to_csv(strActivityFileOutput, index=False)
# Review the dataframe we created.
dfActivities

,id,status,wbs,act_name,duration,start,finish,floor_id
15,NA.FR1010,Not Started,BH.BL-4.115 Nassau1.Floors.4,Layout & Top Track @ Function Room 1,5,4-Apr-16,5-Apr-16,FR1
5,NA.FR1012,Not Started,BH.BL-4.115 Nassau1.Floors.4,Install CMU Walls @ Function Room 1,15,9-May-16,NaN,FR1
2,NA.FR1017,Not Started,BH.BL-4.115 Nassau1.Floors.4,Frame & Core Board Risers/Stairs/Elevators @ F...,10,2-May-16,3-May-16,FR1
19,NA.FR1020,Not Started,BH.BL-4.115 Nassau1.Floors.4,Rough Duct Work @ Function Room 1,10,6-Apr-16,7-Apr-16,FR1
21,NA.FR1030,Not Started,BH.BL-4.115 Nassau1.Floors.4,Rough Mechanical Piping @ Function Room 1,10,6-Apr-16,27-Apr-16,FR1
22,NA.FR1040,Not Started,BH.BL-4.115 Nassau1.Floors.4,Rough Plumbing @ Function Room 1,10,6-Apr-16,7-Apr-16,FR1
20,NA.FR1060,Not Started,BH.BL-4.115 Nassau1.Floors.4,Rough Electrical/FA/BMS @ Function Room 1,10,7-Apr-16,27-Apr-16,FR1
3,NA.FR1070,Not Started,BH.BL-4.115 Nassau1.Floors.4,Frame Walls & Ceilings @ Function Room 1,10,NaN,NaN,FR1
1,NA.FR1100,Not Started,BH.BL-4.115 Nassau1.Floors.4,Firestopping & Inspections @ Function Room 1,5,28-Apr-16,29-Apr-16,FR1
17,NA.FR1120,Not Started,BH.BL-4.115 Nassau1.Floors.4,Prime Paint @ Function Room 1,5,NaN,NaN,FR1


## Relationships:
2. __Relationship/Logic Information :__ Relationship information consists of the realationship / logic information of a project. Each row is a relationship between two activities: a predecessor, and a successor. The columns that make up a relationship are:
    1. __pred_id :__ This is the predecessor activity's ID
    
    2. __succ_id :__ This is the successor activity's ID.

    3. __pred_type :__ This tells the program the type of relationship between the two activities in the row. There are 4 types of relationships:<BR>
       1: FS : the successor cannot _start_ until the predecessor _finishes_.<BR>
       2: FF : the successor cannot _finish_ until the predecessor _finishes_.<BR>
       3: SS : the successor cannot _start_ until the predecessor _starts_.<BR>
       5: SF : the successor cannot _finish_ until the predecessor _starts_. (Considdered innapropriate and will not be used)
    
    4. __pred_status :__ This tells the program the predecessor activity's status. If it has started, or if it has completed. All of the activities we will be making will have a status of "Not Started". This field is basically worthless, but the program requieres it when you are importing an activitiy.

    5. __succ_status :__ same as pred_status, but for the successor.

    6. __lag :__ This field is an integer represents how many days after when the pred_type field says an activity will start or finsh it will actually start or finish.

In [7]:
# This function creates a set of relationships from a list passed in by the user. It consists of two lists:
# 1. lstPred: the list of predecessor floors
# 2. lstSucc: the list of successor floors
# Since the schedule we are creating will be for a multi story building we need to create logic links 
# between floors. The two lists outline how to make these links.
def setRelationshipsList (strFileName, lstPred, lstSucc, strDropTag):
    # Read in the scheudle logic
    dfBaseLogic = pd.read_csv(strFileName ,dtype={'PredID': str, 'SucID': str})

    # We need to make sure that the PredID and SucID strings are the correct length
    # and that they should have leading zeros '0'.
    dfBaseLogic['SucID'] = dfBaseLogic['SucID'].str.zfill(3)
    dfBaseLogic['PredID'] = dfBaseLogic['PredID'].str.zfill(3)

    # Define the logic dataframe
    dfLogic = pd.DataFrame(columns=['pred_id', 'succ_id', 'pred_type', 'pred_status', \
                                    'succ_status', 'lag', 'remove'])
    
    # in the loop below the following line of code is important:
    # dfTemp = pd.DataFrame(dfBaseLogic['Sequence'].replace({0 : lstPred[int(i)], 1 : lstSucc[int(i)]}))
    # this line sets up the sequence column correctly. It sets it up using 2 values:
    #     0: if there is a 0 in the sequence row replace it with the corresponding item in lstPred
    #     1: if there is a 1 in the sequence row replace it with the corresponding item in lstSucc
    # This is important because the 0's go to activities on the same floor as this activity, and 
    # 1's go to activities on the successor floor as this activity.
    
    # this loop creates the dataframe that holds our relationship information
    for i in range(len(lstPred)):
        # assign the correct floor information in the sequence column.
        dfTemp = pd.DataFrame(dfBaseLogic['Sequence'].replace({0 : lstPred[int(i)], 1 : lstSucc[int(i)]}))
        # cache and format the relationship information
        dfNew = pd.DataFrame({
                'pred_id' : 'NA.' + lstPred[int(i)] + dfBaseLogic['PredID'], \
                'succ_id' : 'NA.' + dfTemp['Sequence'] + dfBaseLogic['SucID'], \
                'pred_type' : dfBaseLogic['Type'], \
                'pred_status' : strStatus, \
                'succ_status' : strStatus, \
                'lag' : dfBaseLogic['Lag'], \
                'remove' : dfTemp['Sequence']})
        
        # combine the new dataframe with the exiting one.
        dfLogic = pd.DataFrame(pd.concat([dfLogic,dfNew]))   
    
    # sort the dataframe by the 'pred_id' field
    dfLogic.sort_values('pred_id')
    # reset the index of the dataframe.
    dfLogic.reset_index(inplace=True, drop=True)

    # reset the index of the dataframe
    dfLogic.reset_index(inplace=True, drop=True)
    # drop relationships that will create loops when the schedule is calculated
    dfLogic.drop(dfLogic[dfLogic.succ_id == dfLogic.pred_id].index, inplace=True)
    # reset the index of the dataframe
    dfLogic.reset_index(inplace=True, drop=True)
    # drop relationships that are tagged to be removed
    dfLogic.drop(dfLogic[dfLogic.remove == 'DROP'].index, inplace=True)
    
    # reorder the dataframe and drop the "remove" row because we only needed it for filtering the final product
    dfLogic = dfLogic[['pred_id', 'succ_id', 'pred_type', 'pred_status', 'succ_status', 'lag']]
    
    # return the completed dataframe
    return dfLogic

In [8]:
# this function takes a template of relationships and duplicates them the specified number of times.
# unlike setRelationshipsList which uses lists, this one creates the relationships with a simple loop.
#    intStart: is the starting floor for our relationships
#    intFinish: is the finish floor for our relationships
def setRelationshipsRange(strFileName, intStart, intFinish, intBreakStart = 0, intBreakEnd = 0):
    # Read in the scheudle logic
    dfBaseLogic = pd.read_csv(strFileName ,dtype={'PredID': str, 'SucID': str})

    # We need to make sure that the PredID and SucID strings are the correct length
    # and that they should have leading zeros '0'.
    dfBaseLogic['SucID'] = dfBaseLogic['SucID'].str.zfill(3)
    dfBaseLogic['PredID'] = dfBaseLogic['PredID'].str.zfill(3)

    # Define my dataframe
    dfLogic = pd.DataFrame(columns=['pred_id', 'succ_id', 'pred_type', 'pred_status', \
        'succ_status', 'lag', 'PredFloor', 'SuccFloor'])

    # For each floor we want to create...
    for i in range(intStart, intFinish + 1):
        # cache and format the relationship information
        dfNew = pd.DataFrame({
            'pred_id' : 'NA.' + str(i).zfill(2) + dfBaseLogic['PredID'], \
            'succ_id' : 'NA.' + (i + dfBaseLogic['Sequence']).astype(str).str.zfill(2) + dfBaseLogic['SucID'], \
            'pred_type' : dfBaseLogic['Type'], \
            'pred_status' : strStatus, \
            'succ_status' : strStatus, \
            'lag' : dfBaseLogic['Lag'], \
            'PredFloor' : i, \
            'SuccFloor' : i + dfBaseLogic['Sequence']})
        
        # combine the new dataframe with the exiting one.
        dfLogic = pd.DataFrame(pd.concat([dfLogic,dfNew]))    

    # sort the dataframe by the 'pred_id' field
    dfLogic.sort_values('pred_id')
    
    # reset the index of the dataframe
    dfLogic.reset_index(inplace=True, drop=True)
    # drop rows with links to activities that are linked to intFinish (meaning one floor past the last floor)
    # it is VERY IMPORTANT that this is done AFTER you reset the index.
    dfLogic.drop(dfLogic[dfLogic.SuccFloor > intFinish].index, inplace=True)
    
    # reset the index of the dataframe
    dfLogic.reset_index(inplace=True, drop=True)
    
    # if the user specified a break start
    if intBreakStart > 0 :
        # drop impropper predecessor relationships to activities within the break area
        dfLogic.drop(dfLogic[(dfLogic.PredFloor >= intBreakStart) & (dfLogic.PredFloor < intBreakEnd)].index, inplace=True)
        # reset the index of the dataframe
        dfLogic.reset_index(inplace=True, drop=True)

        # drop impropper successor relationships to activities within the break area
        dfLogic.drop(dfLogic[(dfLogic.SuccFloor >= intBreakStart) & (dfLogic.SuccFloor < intBreakEnd)].index, inplace=True)
        # reset the index of the dataframe
        dfLogic.reset_index(inplace=True, drop=True)
    
    # drop loop creating relationships
    dfLogic.drop(dfLogic[dfLogic.succ_id == dfLogic.pred_id].index, inplace=True)
    
    # reorder the dataframe and drop PredFloor and SuccFloor because we only needed 
    # it for filtering the final product
    dfLogic = dfLogic[['pred_id', 'succ_id', 'pred_type', 'pred_status', 'succ_status', 'lag']]
    
    # return the completed dataframe
    return dfLogic

In [12]:
# This function removes relationships that point to activities that do not exist
def setCleanRelationships(dfLogicList, dfActivitiesList):
    
    # create a mask column that holds which rows in the pred_id column match the 
    # rows in the dfActivities id column
    dfLogicList['drop_mask'] = pd.Series(dfLogicList.pred_id.isin(dfActivitiesList.id))
    # reset the index
    dfLogicList.reset_index(inplace=True, drop=True)
    # Drop the rows outlined in the mask
    dfLogicList.drop(dfLogicList[~dfLogicList.drop_mask].index, inplace=True)
    # reset the index
    dfLogicList.reset_index(inplace=True, drop=True)

    dfLogicList['drop_mask'] = pd.Series(dfLogicList.succ_id.isin(dfActivitiesList.id))
    # reset the index
    dfLogicList.reset_index(inplace=True, drop=True)
    # Drop the rows outlined in the mask
    dfLogicList.drop(dfLogicList[~dfLogicList.drop_mask].index, inplace=True)
    # reset the index
    dfLogicList.reset_index(inplace=True, drop=True)
                               
    # drop the mask column
    dfLogicList.drop(['drop_mask'], axis=1)
    
    # retrun the cleaned dataframe
    return dfLogicList

In [13]:
# I need to create typical floors from the 13th floor to the 44th.
# I will do this by calling my floor creation function defined above. 
dfCreateLogic = setRelationshipsRange(strSourceLogic, 13, 44, 28, 30)

# remove rlationships to activities not in the activities list
dfCreateLogic = setCleanRelationships(dfCreateLogic, dfActivities)

# export the dataframe, without the index
dfCreateLogic.to_csv(strLogicFileOutput, index=False)

# review the dataframe we just exported
# In this run this should be empty
dfCreateLogic.head(5)

,pred_id,succ_id,pred_type,pred_status,succ_status,lag,drop_mask


In [14]:
# Not all predecessor floors will have successor floors. For those activities I will
# add a drop code stored strDrop that will stipulate which rows to drop.
# assign the drop string
strDrop = 'DROP'

# I need to make relationships for floors 02 - 10. There is also a function room on 2 which
# Follows the initial work on 2. that is why you see '02' listed twice. Once for the links
# to the function room ('F3') and once for the 3rd floor ('03').
lstPredecessors = ['02', '02', 'F3', '03', '04', '05', '06', '07', '08', '09', '10']
lstSuccessors = ['F3', '03', strDrop, '04', '05', '06', '07', '08', '09', '10', strDrop]

# call the function to create the successors
dfLogic = setRelationshipsList(strSourceLogic, lstPredecessors, lstSuccessors, strDrop)

# remove rlationships to activities not in the activities list
dfLogic = setCleanRelationships(dfLogic, dfActivities)

# export the dataframe, without the index
dfLogic.to_csv(strLogicFileOutput, index=False)

# review the dataframe we just exported
# In this run this should be empty
dfLogic

,pred_id,succ_id,pred_type,pred_status,succ_status,lag,drop_mask


In [15]:
# I need to make relationships for some other areas in a sister building. As you can see 
# They are not really tied together. Theses are just public spaces in the lower floors.
lstPredecessors = ['FR1', 'FR2', 'PFS', 'PR', 'LNG']
lstSuccessors = ['FR2', 'strDrop', 'PR', strDrop, strDrop]

# call the function to create the successors
dfLogic = setRelationshipsList(strSourceLogic, lstPredecessors, lstSuccessors, strDrop)

# remove rlationships to activities not in the activities list
dfLogic = setCleanRelationships(dfLogic, dfActivities)

# export the dataframe, without the index
dfLogic.to_csv(strLogicFileOutput, index=False)

# review the dataframe we just exported
# In this run there should be a bunch of relationships
dfLogic

,pred_id,succ_id,pred_type,pred_status,succ_status,lag,drop_mask
0,NA.FR1010,NA.FR1020,FS,Not Started,Not Started,0,True
1,NA.FR1017,NA.FR1070,FS,Not Started,Not Started,0,True
2,NA.FR1020,NA.FR1030,SS,Not Started,Not Started,2,True
3,NA.FR1030,NA.FR1040,SS,Not Started,Not Started,2,True
4,NA.FR1060,NA.FR1070,FS,Not Started,Not Started,0,True
5,NA.FR1130,NA.FR1140,FS,Not Started,Not Started,0,True
6,NA.FR1140,NA.FR1150,FS,Not Started,Not Started,0,True
7,NA.FR1150,NA.FR1160,FS,Not Started,Not Started,0,True
8,NA.FR1160,NA.FR1170,FS,Not Started,Not Started,0,True
9,NA.FR1173,NA.FR1170,FS,Not Started,Not Started,0,True
